# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [42]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [43]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼

#取Ticket對Fare做群聚編碼，用mean
mean_df = df.groupby(['Ticket'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Ticket'])['Fare'].apply(lambda x:x.mode()[0]).reset_index()
max_df = df.groupby(['Ticket'])['Fare'].max().reset_index()
median_df = df.groupby(['Ticket'])['Fare'].median().reset_index()
count_df = df.groupby(['Ticket'])['Fare'].count().reset_index()


In [44]:
temp = pd.merge(mean_df, mode_df, how='left', on=['Ticket'])
temp = pd.merge(temp, max_df, how='left', on=['Ticket'])
temp = pd.merge(temp, median_df, how='left', on=['Ticket'])
temp = pd.merge(temp, count_df, how='left', on=['Ticket'])


In [45]:
temp.columns = ['Ticket','Fare_mean','Fare_mode','Fare_max','Fare_median','Fare_count']
temp

,Ticket,Fare_mean,Fare_mode,Fare_max,Fare_median,Fare_count
0,110152,86.5000,86.5000,86.5000,86.5000,3
1,110413,79.6500,79.6500,79.6500,79.6500,3
2,110465,52.0000,52.0000,52.0000,52.0000,2
3,110564,26.5500,26.5500,26.5500,26.5500,1
4,110813,75.2500,75.2500,75.2500,75.2500,1
5,111240,33.5000,33.5000,33.5000,33.5000,1
6,111320,38.5000,38.5000,38.5000,38.5000,1
7,111361,57.9792,57.9792,57.9792,57.9792,2
8,111369,30.0000,30.0000,30.0000,30.0000,1
9,111426,26.5500,26.5500,26.5500,26.5500,1


In [46]:
df_addNew = pd.merge(df, temp, how='left', on=['Ticket'])
df_addNew

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_mean,Fare_mode,Fare_max,Fare_median,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,7.25000,7.2500,7.2500,7.25000,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,71.28330,71.2833,71.2833,71.28330,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,7.92500,7.9250,7.9250,7.92500,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,53.10000,53.1000,53.1000,53.10000,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,8.05000,8.0500,8.0500,8.05000,1
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,8.45830,8.4583,8.4583,8.45830,1
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,51.86250,51.8625,51.8625,51.86250,1
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,21.07500,21.0750,21.0750,21.07500,4
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,11.13330,11.1333,11.1333,11.13330,3
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,30.07080,30.0708,30.0708,30.07080,2


In [47]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [48]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df_addNew.dtypes, df_addNew.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df_addNew = df_addNew[num_features]
df_addNew = df_addNew.fillna(-1)
MMEncoder = MinMaxScaler()
df_addNew.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_max', 'Fare_median', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_max,Fare_median,Fare_count
0,3,22.0,1,0,7.2500,7.2500,7.2500,7.2500,7.2500,1
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833,1
2,3,26.0,0,0,7.9250,7.9250,7.9250,7.9250,7.9250,1
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000,2
4,3,35.0,0,0,8.0500,8.0500,8.0500,8.0500,8.0500,1


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [29]:
# 原始特徵 + 邏輯斯迴歸

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [33]:
df

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
5,3,-1.0,0,0,8.4583
6,1,54.0,0,0,51.8625
7,3,2.0,3,1,21.0750
8,3,27.0,0,2,11.1333
9,2,14.0,1,0,30.0708


In [49]:
# 新特徵 + 邏輯斯迴歸

train_X = MMEncoder.fit_transform(df_addNew)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7016289882888318